# Project 1
## Introduction
The differential equations arising from real-life systems are sometimes "stiff", which make problems especially hard to solve. One definition of a stiff IVP is the following

The initial-value ODE problem is stiff if the step size needed to maintain absolute stability of the forward Euler method is much smaller than the step size needed to represent the solution accurately.

In this project, we propose to analyze some properties of such systems and design specific numerical methods to solve them. We will see for example that classical Runge-Kutta methods are not very efficient solvers in this context. You will focus on three situations of increasing complexity, that will allow you to experiment and discover numerical properties of stiff systems, and understand their underlying difficulties.

You should write a small project report that shows your results, and summarize your findings. For the evaluation I will pay attention to the quality of the interpretations, how you apply the algorithms and the quality of the numerical illustrations. It does not need to be long: I will prefer a short summary that shows that you have understood and analyzed the methods.

## 1 Curtis and Hirschfelder example
We consider the following model, coming from [1]

$$
\left\{\begin{aligned}
d_t y(t) & =\lambda(-y(t)+\cos (t)) \quad \text { with } \lambda>1 \\
y(0) & =y_0
\end{aligned}\right.
$$


The exact solution writes:

$$
y(t)=C e^{-\lambda t}+\frac{\lambda^2 \cos (t)}{1+\lambda^2}+\frac{\lambda \sin (t)}{1+\lambda^2} \quad \text { with } C=y_0-\frac{\lambda^2}{1+\lambda^2} \in \mathbb{R}
$$


The solution of the problem follows a particular dynamic. From an initial state $y_0$, the solution exhibits a short time, fast dynamic of the form $e^{-\lambda t}$ (also called initial time boundary layer), and a slower "steady-state" dynamic of the form $\bar{y}(t)=\cos (t)$ when $\lambda \gg 1$.

## Your tasks
Your first task consists in experimenting the explicit and implicit Euler methods, as well as the explicit midpoint method (2nd order Runge-Kutta). To do so you should first complete the relevant parts of the Python script that has been provided to you.
1. Verify your implementation and identify the stability limits of the explicit methods.
2. What is the drawback of explicit methods for such a problem with both a fast and slow dynamic ? What would you expect when solving more complex problems in term of computational time ?
1
3. For the implicit method, what can you tell about its stability and its quality of approximation ? Do you resolve the boundary layer? What happens if you increase the time step in terms of accuracy?
4. What are the orders of the methods? What is the difference in the convergence plot if you start the simulation further or closer to the steady state solution?

## 2 Brusselator
The Brusselator model is an example of an autocatalytic chemical reaction initiated by [2]. It describes one of the simplest oscillatory chemical reaction. The model can be described by the following mathematical system

$$
\left\{\begin{aligned}
d_t y_1(t) & =1+a y_1^2(t) y_2(t)-(1+b) y_1(t) \\
d_t y_2(t) & =-a y_1^2(t) y_2(t)+b y_1(t) \\
y_1(0) & =1.5 \\
y_2(0) & =3
\end{aligned}\right.
$$

where $(a, b)$ are two parameters. In this section we propose to use the following solvers
1. explicit Euler,
2. implicit Euler,
3. second-order Runge-Kutta,
4. fourth-order Runge-Kutta,
5. third-order embedded Runge-Kutta method
6. the scipy RADAU5 solver implementation, allowing the use of various tolerances and discretizations

## Your tasks
1. Complete the implementation of the fourth-order Runge-Kutta method,
2. Complete the implementation of the embedded method, for which the algorithm is described in the Appendix,
3. Find information on the Radau5 method and explain its general idea.

For the next questions we will use the following parameters

$$
a=1, b=3
$$

1. Analyze the stiffness of the system thanks to the computation of the eigenvalues proposed in the notebook,
2. Experiment the explicit methods with fixed time step and identify their stability limits. Plot your solutions,
3. What is the drawback of explicit methods for such a problem with both a fast and slow dynamic ? What is the phenomena that limits the time step ?
4. Experiment the embedded Runge-Kutta method and explain why the the method is efficient for the problem. Justify the order of the method. What does it bring to have an embedded method ? Is the tolerance as expected?
5. What can you say about the stability of the implicit Euler method? What can you say when the time step is increased in terms of accuracy? What time step do you need to achieve the same quality of resolution as for the Explicit Euler method (when it is stable)?
6. For Radau5, what time steps are used ? Why is this method "overkill", even if it is efficient in terms of accuracy?

## Oregonator
The Oregonator model [3] was obtained in from the chemical reaction of Field, Koros and Noyes [4]. It is similar to the Brusselator and is another example of a chemical autocatalytic reaction. It writes

$$
\left\{\begin{aligned}
d_t y_1 & =y_2-y_1 \\
\varepsilon d_t y_2 & =q y_3-y_3 y_2+y_2\left(1-y_2\right) \\
\mu d_t y_3 & =-q y_3-y_3 y_2+f y_1
\end{aligned}\right.
$$


We will use the following parameters

$$
\varepsilon=10^{-2}, \mu=10^{-5}, f=1, q=2 \times 10^{-4}
$$

with the initial conditions $\left(y_1(0), y_2(0), y_3(0)=(0.5,0,1200)\right.$, and $t \in[0,30]$. This model exhibits a strong stiffness. You will use the Euler explicit, Runge Kutta 2, Runge Kutta 4, and the Radau5 methods.

## Your tasks
1. Looking at the dynamic, propose a qualitative analysis of the system stiffness,
2. Try to solve the system with an explicit method and show that the stability constraint to resolve the dynamic implies a very small time step. What is the drawback of explicit methods for such a problem with both a fast and slow dynamic ?
3. For Radau5, what are the used time steps as a function of the tolerance? Compare with the time steps from the explicit methods. Why is the method optimal for this problem in terms of stability and accuracy compared to the case of the Brusselator system ?

## Appendix: adaptive time stepping strategy - embedded Runge-Kutta
The idea is to adapt the time step to the local dynamics in order to provide an efficient integration strategy. The user should provide a tolerance and the adaptation has to rely on an error estimate and should produce a time step so that the local error estimate is below the given tolerance. The idea to provide such an error estimate is to combine two methods with different orders such that the difference between the two is a conservative error estimate. However, building up a lower order method from a given one, should not result in an important increase of the computational effort. Thus the idea of embedded methods in order to minimize the number of function evaluations, and consequently the computational effort. We will rely on the a Runge-Kutta of order 4 with 4 stages, known as the $3 / 8$ rule and will construct a 3 rd order embedded method. Starting from the $k_i$ values, $i=\{1,2,3,4\}$, obtained

**insert the table here**

Table 1: Butcher tableau for the $3 / 8$ rule
through the previous Runge-Kutta method, we will build a $s+1$ stages method of order 3

$$
\widehat{y}_1=y_0+\Delta t\left(\widehat{b}_1 k_1+\ldots, \widehat{b}_s k_s+\widehat{b}_{s+1} f\left(t_1, y_1\right)\right)
$$

where the last point has to be evaluated anyway and $\widehat{b}_{s+1}$ provides more flexibility. The order conditions are obtained using the usual way, except that we have another stage here $\left(a_{s+1, i}=b_i, i=1, \ldots, s\right)$ and yield four
equations:

$$
\begin{aligned}
\widehat{b}_1+\widehat{b}_2+\widehat{b}_3+\widehat{b}_4+\widehat{b}_5 & =1 \\
\widehat{b}_2 c_2+\widehat{b}_3 c_3+\widehat{b}_4+\widehat{b}_5 & =1 / 2 \\
\widehat{b}_2 c_2^2+\widehat{b}_3 c_3^2+\widehat{b}_4+\widehat{b}_5 & =1 / 3 \\
\widehat{b}_3 a_{32} c_2+\widehat{b}_4\left(a_{42} c_2+a_{43} c_3\right)+\widehat{b}_5 / 2 & =1 / 6
\end{aligned}
$$


We have five unknowns and four equations. We choose $\widehat{b}_5=1 / 6$ and obtain :

$$
\widehat{b}_1=2 b_1-1 / 6, \quad \widehat{b}_2=2\left(1-c_2\right) b_2, \quad \widehat{b}_3=2\left(1-c_3\right) b_3, \quad \widehat{b}_4=0
$$


Thus, using a time step $\Delta t$, we obtain :

$$
y_1-\widehat{y}_1=y_1-y\left(t_0+\Delta t\right)+y\left(t_0+\Delta t\right)-\widehat{y}_1=O\left((\Delta t)^{p+1}\right)+O\left((\Delta t)^{\hat{p}+1}\right) \approx C(\Delta t)^{\hat{p}+1}
$$


The optimal time step $\Delta t_{\mathrm{opt}}$ is given by the fact that

$$
\mathrm{Tol} \approx C\left(\Delta t_{\mathrm{opt}}\right)^{\hat{p}+1}
$$

so that by eliminating the constant C between the last two equations we get

$$
\Delta t_{\mathrm{opt}}=0.9 \Delta t \sqrt[\hat{p}+1]{\frac{\mathrm{Tol}}{\left\|y_1-\widehat{y}_1\right\|}}
$$

where the 0.9 factor is called a security factor. For practical purposes and robustness of the method, it is standard to replace the last evaluation of the time step by

$$
\Delta t_{\mathrm{opt}}=\Delta t \min \left(5, \max \left(0.2,0.9 \sqrt[\hat{p}+1]{\frac{\mathrm{Tol}}{\left\|y_1-\widehat{y}_1\right\|}}\right)\right)
$$

and the norm is taken as a mix of the relative and absolute $l^2$ norm :

$$
\left\|y_1-\widehat{y_1}\right\|=\sqrt{\frac{1}{m} \sum_{j=1}^m\left(\frac{y_{j 1}-\widehat{y}_{j 1}}{1+\max \left(\left|y_{j 0}\right|,\left|y_{j 1}\right|\right)}\right)^2}
$$
In order to be clear on what has been coded in the notebook, here is the algorithm, which starts from an initial condition $y_0$, a tolerance Tol and a given time step $\Delta t_1$ at $n=1$ :
```
Algorithm 1: Automatic selection of the adaptive time step
    A) With the current time step $\Delta t_n$ and from $y_{n-1}$ evaluate $y_n, \widehat{y}_n$ and err $=\left\|y_n-\widehat{y}_n\right\|$ as well as $\Delta t_{\mathrm{opt}, n}$
        using the definitions above
    B) Advance in time or adapt the time step
    if err $\leq$ Tol then
        (the time step is accepted)
        $t_{n+1}:=t_n+\Delta t_n$
        $\Delta t_{n+1}=\min \left(\Delta t_{\mathrm{opt}, n}, t_{\text {end }}-\Delta t_n\right)$
        the new state of the system if taken as $y_n, n:=n+1$
    else
        (the time step is rejected) $\Delta t_n=\Delta t_{\mathrm{opt}, n}$
    end
    C) If the current time is $t_n=t_{\text {end }}$ the simulation is over, else we start again at A)
```